In [ ]:
import geopandas as gpd 
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Import the voronoi shape of India
import dssg.dataio.osm_to_voronoi_mapping as ovm
data_dir = os.environ.get("DATA_DIR")
india_voronoi_gpd = gpd.read_file(data_dir + "voronoi3_clip/voronoi3_clip.shp")

In [ ]:
india_voronoi_gpd.head()

In [ ]:
import dssg.dataio.osm_data_extraction as ode
india_shape = os.environ.get("DATA_DIR") + "/gadm36_shp/gadm36_IND_2.shp"

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_dataframe
import matplotlib.pyplot as plt 
%matplotlib inline
india_gpd = gpd.read_file(india_shape)
#Extract a district from gadm36_IND_2.shp file
araria_gdf = ode.extract_district_dataframe(india_gpd, 'Araria')

In [ ]:
print(open('../profile/extract_district_dataframe', 'r').read())

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_voronoi_clipped
#Extract the GeoDataFrame of the voronoi clipped to the district boundary
araria_voronoi_gpd_clipped = ovm.extract_district_voronoi_clipped(india_voronoi_gpd, araria_gdf)

In [ ]:
print(open('../profile/extract_district_voronoi_clipped', 'r').read())

In [ ]:
ovm.plot_district_voronoi(araria_voronoi_gpd_clipped, "Araria")

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_voronoi_wo_clipping
araria_voronoi_joined = ovm.extract_district_voronoi_wo_clipping(india_voronoi_gpd, araria_gdf)

In [ ]:
print(open('../profile/extract_district_voronoi_wo_clipping', 'r').read())

In [ ]:
ovm.plot_district_voronoi(araria_voronoi_joined, "Araria")

In [ ]:
#Read in the predictions for araria:
araria_pred_file = os.environ.get("DATA_DIR") + "/osm/araria_results_plot.csv" 
import pandas as pd 
araria_pred = pd.read_csv(araria_pred_file)

In [ ]:
araria_pred.head()

In [ ]:
araria_voronoi_joined.head()

In [ ]:
araria_pred.shape, araria_voronoi_joined.shape

In [ ]:
araria_merged = pd.merge(araria_pred, araria_voronoi_joined, on="DHSCLUST")

In [ ]:
araria_merged = araria_merged.drop(['DHSCC', 'DHSYEAR', 'LATNUM', 'LONGNUM', 'GID_2c', 'index_right', 'NAME_2'], axis=1)

In [ ]:
araria_merged.head()

In [ ]:
type(araria_merged)

In [ ]:
araria_merged_gdf = gpd.GeoDataFrame(araria_merged)

In [ ]:
#Araria plot real wealth with voronoi cells:
import matplotlib.pyplot as plt 
import PyQt5
import mapclassify as mc
%matplotlib qt
fig, ax = plt.subplots(1,1)
araria_merged_gdf.plot(column='real', cmap='cubehelix', ax=ax, legend=True, legend_kwds={'label':'Real Wealth Distribution by DHS Cluster in Araria','orientation':"horizontal"})

In [ ]:
fig1, ax1 = plt.subplots(1,1)
araria_merged_gdf.plot(column='preds', cmap='cubehelix', ax=ax1, legend=True, legend_kwds={'label':'Predicted Wealth Distribution by DHS Cluster in Araria','orientation':"horizontal"})